In [4]:
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict

In [3]:
main_dir = 'm5out_Bubblesort'
output_csv = 'gem5_simulation_results.csv'
fields_to_extract = [
    "simSeconds",
    "system.cpu.cpi",
    "system.cpu.ipc",
    "system.cpu.numIssuedDist::mean",
    "system.cpu.issueRate",
    "system.cpu.decode.blockedCycles",
    "system.cpu.fetch.icacheStallCycles",
    "system.cpu.commit.branchMispredicts",
    "system.cpu.branchPred.condIncorrect",
]

# List to store extracted data
data = []

# Traverse each subdirectory in the main directory
for subdir in os.listdir(main_dir):
    subdir_path = os.path.join(main_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        stats_file_path = os.path.join(subdir_path, 'stats.txt')
        
        # Check if stats.txt exists in the subdirectory
        if os.path.exists(stats_file_path):
            with open(stats_file_path, 'r') as f:
                # Dictionary to store the second occurrence of each field
                values = {field: None for field in fields_to_extract}
                occurrences = {field: 0 for field in fields_to_extract}
                
                # Read through the file line by line
                for line in f:
                    # Check if the line contains any of the desired fields
                    for field in fields_to_extract:
                        if line.startswith(field):
                            # Only capture the second occurrence
                            occurrences[field] += 1
                            if occurrences[field] == 2:
                                # Split line by whitespace and get the second part (value)
                                value = line.split()[1]
                                values[field] = value

                # Only add to data if we found the second occurrence of all fields
                if all(values[field] is not None for field in fields_to_extract):
                    # Append the data: folder name and extracted values
                    data.append([subdir] + [values[field] for field in fields_to_extract])

# Write the data to a CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header
    writer.writerow(['Configuration'] + fields_to_extract)
    # Write the data rows
    writer.writerows(data)

print(f"Data has been written to {output_csv}")

Data has been written to gem5_simulation_results.csv


In [5]:
main_dir = 'm5out_Bubblesort'

# Output CSV file for absolute and relative values
output_csv = 'gem5_simulation_results_relative.csv'

# Fields to extract
fields_to_extract = [
    "simSeconds",
    "system.cpu.cpi",
    "system.cpu.ipc",
    "system.cpu.numIssuedDist::mean",
    "system.cpu.issueRate",
    "system.cpu.decode.blockedCycles",
    "system.cpu.fetch.icacheStallCycles",
    "system.cpu.commit.branchMispredicts",
    "system.cpu.branchPred.condIncorrect",
]

# List to store extracted absolute data
absolute_data = []

# Traverse each subdirectory in the main directory
for subdir in os.listdir(main_dir):
    subdir_path = os.path.join(main_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        stats_file_path = os.path.join(subdir_path, 'stats.txt')
        
        # Check if stats.txt exists in the subdirectory
        if os.path.exists(stats_file_path):
            with open(stats_file_path, 'r') as f:
                values = {field: None for field in fields_to_extract}
                occurrences = {field: 0 for field in fields_to_extract}
                
                # Read through the file line by line
                for line in f:
                    for field in fields_to_extract:
                        if line.startswith(field):
                            occurrences[field] += 1
                            if occurrences[field] == 2:
                                value = float(line.split()[1])
                                values[field] = value

                # Only add to data if we found the second occurrence of all fields
                if all(values[field] is not None for field in fields_to_extract):
                    absolute_data.append([subdir] + [values[field] for field in fields_to_extract])

# Calculate reference values (mean of each field)
reference_values = {}
for i, field in enumerate(fields_to_extract, start=1):
    field_values = [row[i] for row in absolute_data]
    reference_values[field] = sum(field_values) / len(field_values) if field_values else 1  # Avoid division by zero

# Write absolute and relative values to a CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header
    writer.writerow(['Configuration'] + fields_to_extract + [f"relative_{field}" for field in fields_to_extract])

    # Write each row of absolute and relative values
    for row in absolute_data:
        config_name = row[0]
        abs_values = row[1:]
        rel_values = [abs_val / reference_values[field] for abs_val, field in zip(abs_values, fields_to_extract)]
        writer.writerow([config_name] + abs_values + rel_values)

print(f"Data with relative values has been written to {output_csv}")


Data with relative values has been written to gem5_simulation_results_relative.csv


In [6]:
main_dir = 'm5out_Bubblesort'

# Fields to extract
fields_to_extract = [
    "simSeconds",
    "system.cpu.cpi",
    "system.cpu.ipc",
    "system.cpu.numIssuedDist::mean",
    "system.cpu.issueRate",
    "system.cpu.decode.blockedCycles",
    "system.cpu.fetch.icacheStallCycles",
    "system.cpu.commit.branchMispredicts",
    "system.cpu.branchPred.condIncorrect",
]

# Dictionary to store extracted absolute data
absolute_data = defaultdict(dict)

# Traverse each subdirectory in the main directory
for subdir in os.listdir(main_dir):
    subdir_path = os.path.join(main_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        stats_file_path = os.path.join(subdir_path, 'stats.txt')
        
        # Check if stats.txt exists in the subdirectory
        if os.path.exists(stats_file_path):
            with open(stats_file_path, 'r') as f:
                values = {field: None for field in fields_to_extract}
                occurrences = {field: 0 for field in fields_to_extract}
                
                # Read through the file line by line
                for line in f:
                    for field in fields_to_extract:
                        if line.startswith(field):
                            occurrences[field] += 1
                            if occurrences[field] == 2:
                                value = float(line.split()[1])
                                values[field] = value

                # Only add to data if we found the second occurrence of all fields
                if all(values[field] is not None for field in fields_to_extract):
                    absolute_data[subdir] = values

# Calculate reference values (mean of each field)
reference_values = {}
for field in fields_to_extract:
    field_values = [values[field] for values in absolute_data.values()]
    reference_values[field] = sum(field_values) / len(field_values) if field_values else 1  # Avoid division by zero

# Dictionary to store relative values
relative_data = defaultdict(dict)

# Calculate relative values
for config, values in absolute_data.items():
    relative_data[config] = {field: values[field] / reference_values[field] for field in fields_to_extract}

print(relative_data)

defaultdict(<class 'dict'>, {'2bit_96_6': {'simSeconds': 1.0608207716315825, 'system.cpu.cpi': 1.0608012571398695, 'system.cpu.ipc': 0.8651570076765193, 'system.cpu.numIssuedDist::mean': 0.9959491663227158, 'system.cpu.issueRate': 0.9963422957746384, 'system.cpu.decode.blockedCycles': 1.021974121136783, 'system.cpu.fetch.icacheStallCycles': 1.2789163377061519, 'system.cpu.commit.branchMispredicts': 2.4660523219987676, 'system.cpu.branchPred.condIncorrect': 2.466202280596257}, 'bimode_96_6': {'simSeconds': 0.973896009031144, 'system.cpu.cpi': 0.9739087444010408, 'system.cpu.ipc': 0.9423469853350165, 'system.cpu.numIssuedDist::mean': 1.0374120112494907, 'system.cpu.issueRate': 1.037669324131828, 'system.cpu.decode.blockedCycles': 0.8630763809760847, 'system.cpu.fetch.icacheStallCycles': 1.1765251126651026, 'system.cpu.commit.branchMispredicts': 1.9157402946613633, 'system.cpu.branchPred.condIncorrect': 1.9154580801308747}, 'large': {'simSeconds': 0.6626602032007437, 'system.cpu.cpi': 0.6